# Part 1: About neural data

In [ ]:
! pip install mne
import mne
import matplotlib.pyplot as plt
import mne.viz

import numpy as np
from scipy.stats import ttest_ind
import copy

## Loading the data and the Epochs object

First we need to download the data from GitHub with the following function:

In [ ]:
import requests
def download_file(url, outfile=None):
    if not outfile:
        outfile = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(outfile, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)


In [ ]:
download_file('https://github.com/fma0/AMLD/blob/main/902-P.fif?raw=true', outfile='902-P.fif')

Then we load the .fif file containing the epoched EEG recording of participant 902. 

In [ ]:
data_file = '902-P'

In [ ]:
epochs = mne.read_epochs(data_file + '.fif', verbose='error')

In [ ]:
epochs

The output of epochs tells us, that we have 154 events, i.e. single trials. Out of which 27 are called *Novel* and 127 *Standard*. This data is from a patient exposed to two different types of audtiory stimulations. 
We can get all of the trials for the *Standard* condition with:

In [ ]:
epochs['Standard']

The epoch datastructure of mne has an attribute '[info](https://mne.tools/stable/generated/mne.Info.html#mne.Info)', containing information on the sample frequency, the names of the electrodes, filtering information, etc.

In [ ]:
epochs.info

We find for example the sample frequecy as follows:

In [ ]:
sfreq = epochs.info['sfreq']
print(f'The smaple frequency is {sfreq} Hz')

And the recorded channels / electrodes  with:

In [ ]:
ch_names = epochs.info['ch_names']
print(f'The recording contains the following channels: {ch_names}')

### Exercise 1:

How many electrodes were recorded? 

**Tip:** Either take the length of ch_names, or check the epochs.info above.

What is the index of 'Cz'? 

**Tip**: Define channel_Cz such that: ch_names[channel_Cz] = 'Cz'.

The epochs object also gives you a numpy array with all the collected timepoint per trial

In [ ]:
times = epochs.times
print(times)

You can find additional information for the epochs object here: [Epoch](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs)

Now we will look at how to extract the data and some of the functions provided by mne for plotting

In [ ]:
data = epochs.get_data()

The data object is 3 dimensional.
### Exercise 2:
In what order are the dimensions time, channels and number of trials stored in the epochs object?

We can plot some single-trail exemplar traces for electrode 0 with the following code snippet:

In [ ]:
plt.plot(times, data[14:20,0,:].T)
plt.title("Exemplar single-trial epoched data, for electrode 0")
plt.ylabel('Volts')
plt.xlabel('Time (s)')
plt.show()

### Exercise 3:
Plot the last 5 single-trial traces for electrode Cz

**Tipp**: We found out above at which index the electrode Cz is located at (Exercise 1)

## The Evoked Object

In neuroscience we oftentime represent the data as averages over multiple trials. In this way we get rid of background noise and can see a neural response clearer. In mne this object is called 'evoked' and can be generated with epochs.average():

In [ ]:
evoked = epochs.average()

In [ ]:
evoked_data = evoked._data
shape_evoked = evoked_data.shape
print(f'The evoked has the shape: {shape_evoked} (Channels x Times)')

### Exercise 4:

1st plot the evoked trace for electrode Cz and then the averaged traces for all channels simultaneously

This can be done easier with a function from mne:

In [ ]:
evoked.plot();

### Exercise 5:
Plot the averaged traces for both conditions available in the data (*Standard* & *Novel*). These types of figures are called ERP's (event-related potential)

### Exercise 6:
With matplotlib (as in Exercise 4) plot at the Cz electrode the traces for both conditions (*Standard* and *Novel*) in the same graph

In [ ]:
ep_std = epochs['Standard']
ep_nov = epochs['Novel']

fig, ax = plt.subplots(figsize=(10, 5), dpi=300)
ax.set_xlabel('Time Instances')
ax.set_ylabel('Volt')

ax.plot(..., color='blue', label='Standard')
ax.plot(..., color='red', label='Novel')

legend = ax.legend(loc='upper right', shadow=True, fontsize='medium')
plt.title('ERP of different conditions')
plt.show()

## Topographic Maps
We can use a function from mne to plot the averaged neural data in a topographic map representation. We can specify one or multiple timepoints at which we would like to visualize the activation projected to the scalp.

In [ ]:
timepoints = np.arange(0, 0.51, 0.1)
epochs.average().plot_topomap(timepoints, ch_type='eeg'); 

### Exercise 7:

Use the plot_topomap function to visualize the activation for both conditions

## Reference
This subchapter focuses on references and how much this could influence your data. 
### Exercise 8:
1. Make two copies of the original epochs object (copy.deepcopy(epochs)), called original_1 and original_2
2. Set the reference once to 'average' and once to the electrode 'Fz' (epochs.set_eeg_reference(ref)
3. Visualize the mean ERP for both of these cases (see Exercise 5)

In [ ]:
original_1 = 
original_2 = 

Observe how different the data looks in these two representations. 

We can also look at the difference in each electrode seperately, here we look at CPz:

In [ ]:
references = ['original', 'Oz']
channel_CPz = ch_names.index('CPz')

#Data with original reference
st_1 = original_1['Standard']
nv_1 = original_1['Novel']

#Data with Oz as the reference channel
st_2 = original_2['Standard']
nv_2 = original_2['Novel']

fig, ax = plt.subplots(figsize=(10, 5),  dpi=300)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Volt') 
ax.set_title('ERP values of novel and standard conditions with original reference')
ax.plot(st_1.average().get_data()[channel_CPz,:], color='blue', label='Standard')
ax.plot(nv_1.average().get_data()[channel_CPz,:], color='red', label='Novel')
ax.legend()
plt.show()


fig, ax = plt.subplots(figsize=(10, 5),  dpi=300)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Volt') 
ax.set_title('ERP values of novel and standard conditions with TP8 channel as reference')
ax.plot(st_2.average().get_data()[channel_CPz,:], color='blue', label='Standard')
ax.plot(nv_2.average().get_data()[channel_CPz,:], color='red', label='Novel')
plt.show()

The following exampe is not scientifically correct and this should not be used. It's purpouse is purely illustrative.

Let's test at every time point and every electrode, if the signal of the *Standard* and *Novel* trials are significantly different, with a simple t-test over all single trials. E.g. for electrode 0 and time point 0 we would test:

In [ ]:
ttest_ind(original_1['Standard'].get_data()[:,0,0], original_1['Novel'].get_data()[:,0,0])

If the p-value (at the second position) is bellow 0.05 we will consider it significant. 
Let's copy the evoked structure and replace the data with information about the significance of an electrode, i.e. we replace for the above example 
test_ep_1[0,0] with 0, as it was non significant.

We loop over all time points and electrodes, for the two different references:

In [ ]:
significant_ep_1 = copy.deepcopy(evoked)

for i in range(len(times)):
    for ch in range(len(ch_names)):
        if ttest_ind(original_1['Standard'].get_data()[:,ch,i], 
                     original_1['Novel'].get_data()[:,ch,i])[1] < 0.05:
            significant_ep_1._data[ch, i] = 1
        else:
            significant_ep_1._data[ch, i] = 0

In [ ]:
significant_ep_2 = copy.deepcopy(evoked)

for i in range(301):
    for ch in range(60):
        if ttest_ind(original_2['Standard'].get_data()[:,ch,i], 
                     original_2['Novel'].get_data()[:,ch,i])[1] < 0.05:
            significant_ep_2._data[ch, i] = 1
        else:
            significant_ep_2._data[ch, i] = 0

### Exercise 9:
Plot the topographic map representation of the two evoked objects, containing information about the significance of electrodes and time points